Begin The Data Scraping

In [102]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

url ='https://cnfstore.com/sale?cat=77&p={}&product_list_limit=30'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}
listBrand = []
listProdName =[]
listVolume = []
listType = []
listOldPrice = []
listNewPrice = []
listDiscount = []

count_page = 0
count_item=0
for page in range (1,14):
    count_page+=1
    print('scraping page :',count_page)
    req = requests.get(url.format(page), headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    items = soup.findAll('div', 'product-card-wrapper')

    for it in items:
        count_item+=1
        # extract the brand name
        try: brand = it.find('span', 'product-card-brand').text
        except: brand = '-'
        listBrand.append(brand)

        # extract the product name
        prod_name = it.find('a', 'product-card-name').text
        listProdName.append(prod_name)

        # extract the parfume total volume
        match = re.findall(r'(\d+)\s?ml', prod_name)
        total_volume = 0
        for vol in match:
            quantity = int(re.search(r'\d+', vol).group())
            total_volume += quantity
        listVolume.append(total_volume)

        # extract the parfume type
        try: 
            match = re.search(r'ED[PT]', prod_name, flags=re.IGNORECASE)
            type = match.group()
        except: type  = '-'
        listType.append(type)
        
        # extract the old price
        try: 
            old_price = it.find('div','product-card-price').find('div','old-price').find('span','price').text
            old_price = int (old_price.replace('Rp','').replace(' ','').replace('.',''))
        except: old_price = int(0)
        listOldPrice.append(old_price)

        # extract the new price
        try: 
            new_price = it.find('div','product-card-price').find('div','special-price').find('span','price').text
            new_price = int (new_price.replace('Rp','').replace(' ','').replace('.',''))
        except: new_price = int(0)
        listNewPrice.append(new_price)

        # extract the discount
        try: 
            discount = it.find('div','product-card-price').find('div','old-price').find('span','sale-percent').text
            discount = int (discount.replace('-','').replace('%',''))
        except: discount = int(0)
        listDiscount.append(discount)

print(count_item)



scraping page : 1
scraping page : 2
scraping page : 3
scraping page : 4
scraping page : 5
scraping page : 6
scraping page : 7
scraping page : 8
scraping page : 9
scraping page : 10
scraping page : 11
scraping page : 12
scraping page : 13
370


Create a Data Frame

In [103]:
sale_product_dict = {
    'Brand': listBrand,
    'Product Name': listProdName,
    'Size (ml)': listVolume,
    'Type': listType,
    'Old Price (IDR)': listOldPrice,
    'Special Price (IDR)': listNewPrice,
    'Discount Percentage': listDiscount
}

# create a dataframe
df= pd.DataFrame(sale_product_dict, columns=['Brand','Product Name', 'Size (ml)', 'Type', 'Old Price (IDR)', 'Special Price (IDR)', 'Discount Percentage'])
df.head()

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage
0,LOUIS VAREL,Louis Varel My Dream EDP 100 ml,100,EDP,620000,434000,30
1,DOLCE & GABBANA,Dolce & Gabbana Light Blue Intense Man EDP 100 ml,100,EDP,0,0,0
2,KORLOFF,Miss Korloff EDP 88 ml,88,EDP,1690000,1521000,10
3,HUGO BOSS,Hugo Boss Scent Le Parfum Men 100 ml,100,-,1680000,1293600,23
4,OULLU,Oullu Umbra EDP 15 ml,15,EDP,189000,160650,15


In [104]:
# save raw dataframe
df.to_json("C&F_sale_fragrances_raw.json",indent=4)

Begin Data Cleaning

In [105]:
# check for duplicate data
print('Total duplicate data:',df.duplicated().sum())

Total duplicate data: 111


In [106]:
df[df['Product Name'].duplicated()].sort_values('Product Name', ascending=True).head(20)

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage
296,ANGEL SCHLESSER,Angel Schlesser Instant Vibrant Sandalwood EDT...,100,EDT,990000,792000,20
331,ARMAND BASI,Armand Basi L'Eau Pour Homme Vetiver EDT 125 ml,125,EDT,1290000,903000,30
241,-,Avicenna 100 ml + Yves Rocher Body Lotion 390 ML,100,-,0,0,0
184,-,Avicenna EDP 50 ml + Anessa Day Serum Set Free...,50,EDP,0,0,0
250,-,Avicenna EDP 50 ml + Anessa New Perfect UV Sun...,50,EDP,0,0,0
298,AVICENNA,Avicenna Euphoric (New) EDP 100 ml,100,EDP,379000,284250,25
288,AVICENNA,Avicenna Freedom (New) EDT 100 ml,100,EDT,379000,284250,25
197,AVICENNA,Avicenna Freedom (New) EDT 100 ml,100,EDT,379000,284250,25
340,AVICENNA,Avicenna Miracle EDP 50 ml,50,EDP,259000,194250,25
334,AVICENNA,Avicenna VIP Women (New) EDP 100 ml,100,EDP,399000,299250,25


In [107]:
# drop duplicate data
df.drop_duplicates(inplace=True)
print('Total duplicate data:',df.duplicated().sum())

Total duplicate data: 0


In [108]:
# check for available products
# out of stock products are indicated with 0 value for both old price and special price
df1 = df[(df['Old Price (IDR)'] != 0) & (df['Special Price (IDR)'] != 0)]
print('Available products:', df1.shape[0])


Available products: 230


In [109]:
# cleaning data from "out of stock" products
df = df[(df['Old Price (IDR)'] != 0) & (df['Special Price (IDR)'] != 0)]
print('Total data rows:',df.shape[0])
df.head(10)

Total data rows: 230


,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage
0,LOUIS VAREL,Louis Varel My Dream EDP 100 ml,100,EDP,620000,434000,30
2,KORLOFF,Miss Korloff EDP 88 ml,88,EDP,1690000,1521000,10
3,HUGO BOSS,Hugo Boss Scent Le Parfum Men 100 ml,100,-,1680000,1293600,23
4,OULLU,Oullu Umbra EDP 15 ml,15,EDP,189000,160650,15
5,CRISTIANO RONALDO,Cristiano Ronaldo CR7 Origins EDT 100 ml,100,EDT,1180000,826000,30
6,MONTBLANC,Montblanc Emblem EDT 100 ml,100,EDT,1400000,1120000,20
7,ST. HILAIRE,St. Hilaire Private Blue EDP 100 ml,100,EDP,530000,371000,30
9,PERFUME REPUBLIC,Perfume Republic Break A Leg EDT 100 ml,100,EDT,249000,161850,35
10,MARCO SERUSSI,Marco Serussi The Man Intense,0,-,1030000,824000,20
11,MUGLER,Mugler Alien Goddess EDP 60 ml,60,EDP,1680000,1327200,21


In [110]:
# display the products that have a type which is neither EDP nor EDT
df[(df['Type'] != 'EDP') & (df['Type'] != 'EDT')]

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage
3,HUGO BOSS,Hugo Boss Scent Le Parfum Men 100 ml,100,-,1680000,1293600,23
10,MARCO SERUSSI,Marco Serussi The Man Intense,0,-,1030000,824000,20
19,BEVERLYS SECRET,Beverly's Secret Sweet Seduction Fragrance Mis...,250,-,150000,90000,40
29,BEVERLYS SECRET,Beverly's Secret Exotic Dream Fragrance Mist 2...,250,-,150000,90000,40
72,BEVERLYS SECRET,Beverly's Secret Vanilla Sparkle Fragrance Mis...,250,-,150000,90000,40
81,BEVERLYS SECRET,Beverly's Secret Cherry Blossom Fragrance Mist...,250,-,150000,90000,40
83,MARCO SERUSSI,Marco Serussi The Lady,0,-,1030000,824000,20
116,YVES ROCHER,Yves Rocher Body & Hair Mist - Vanilla 100 ml,100,-,219000,164250,25
122,YVES ROCHER,Yves Rocher Body & Hair Mist - Olive 100 ml,100,-,219000,164250,25
175,YVES ROCHER,Yves Rocher Body & Hair Mist - Mango 100 ml,100,-,219000,164250,25


In [111]:
# change all '-' type value with EDC|Body & Hair Mist|Toner|Fragrance Mist corresponding with the product name
df.loc[df['Type'] == '-', 'Type'] = df.loc[df['Type'] == '-', 'Product Name'].str.extract('(EDC|Body & Hair Mist|Toner|Fragrance Mist)', expand=False).fillna('-')
df[(df['Type'] != 'EDP') & (df['Type'] != 'EDT')]

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage
3,HUGO BOSS,Hugo Boss Scent Le Parfum Men 100 ml,100,-,1680000,1293600,23
10,MARCO SERUSSI,Marco Serussi The Man Intense,0,-,1030000,824000,20
19,BEVERLYS SECRET,Beverly's Secret Sweet Seduction Fragrance Mis...,250,Fragrance Mist,150000,90000,40
29,BEVERLYS SECRET,Beverly's Secret Exotic Dream Fragrance Mist 2...,250,Fragrance Mist,150000,90000,40
72,BEVERLYS SECRET,Beverly's Secret Vanilla Sparkle Fragrance Mis...,250,Fragrance Mist,150000,90000,40
81,BEVERLYS SECRET,Beverly's Secret Cherry Blossom Fragrance Mist...,250,Fragrance Mist,150000,90000,40
83,MARCO SERUSSI,Marco Serussi The Lady,0,-,1030000,824000,20
116,YVES ROCHER,Yves Rocher Body & Hair Mist - Vanilla 100 ml,100,Body & Hair Mist,219000,164250,25
122,YVES ROCHER,Yves Rocher Body & Hair Mist - Olive 100 ml,100,Body & Hair Mist,219000,164250,25
175,YVES ROCHER,Yves Rocher Body & Hair Mist - Mango 100 ml,100,Body & Hair Mist,219000,164250,25


In [112]:
# check for any '-' value left in the Type column
df[(df['Type'] == '-')]

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage
3,HUGO BOSS,Hugo Boss Scent Le Parfum Men 100 ml,100,-,1680000,1293600,23
10,MARCO SERUSSI,Marco Serussi The Man Intense,0,-,1030000,824000,20
83,MARCO SERUSSI,Marco Serussi The Lady,0,-,1030000,824000,20


In [113]:
# I do a further investigation in C&F website to determine the kind of unknown fragrances' type
df.loc[df['Product Name'] == 'Marco Serussi The Man Intense', 'Type'] = 'EDT'
df.loc[df['Product Name'] == 'Marco Serussi The Lady', 'Type'] = 'EDP'
df.loc[df['Product Name'] == 'Hugo Boss Scent Le Parfum Men 100 ml', 'Type'] = 'EDP'

In [114]:
# double check '-' values for ''Type' column
df[df['Type'] == '-']

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage


In [115]:
# check for '0' value in 'Brand' column
df[df['Brand']=='-']

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage


In [116]:
# check for '0' value in 'Size (ml)' column
df[df['Size (ml)']==0]

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage
10,MARCO SERUSSI,Marco Serussi The Man Intense,0,EDT,1030000,824000,20
83,MARCO SERUSSI,Marco Serussi The Lady,0,EDP,1030000,824000,20


In [117]:
# I do a further investigation in Google to determine the volume of unknown fragrances' size
df.loc[df['Product Name'] == 'Marco Serussi The Man Intense', 'Size (ml)'] = 100
df.loc[df['Product Name'] == 'Marco Serussi The Lady', 'Size (ml)'] = 90


In [118]:
# double-check for MARCO SERUSSI's fragrances details
df[df['Brand']=='MARCO SERUSSI']

,Brand,Product Name,Size (ml),Type,Old Price (IDR),Special Price (IDR),Discount Percentage
10,MARCO SERUSSI,Marco Serussi The Man Intense,100,EDT,1030000,824000,20
83,MARCO SERUSSI,Marco Serussi The Lady,90,EDP,1030000,824000,20
224,MARCO SERUSSI,Marco Serussi Harmony Intense EDP 100 ml,100,EDP,990000,792000,20
310,MARCO SERUSSI,Marco Serussi The Lady Intense EDP 90 ml,90,EDP,1030000,824000,20
357,MARCO SERUSSI,Marco Serussi The Man Trust EDT 100 ml,100,EDT,1030000,824000,20


In [119]:
# save cleaned data frame
df.to_json("C&F_sale_fragrances_cleaned.json",indent=4)

At this moment, I forgot add a column to identify each row. Therefore, I decide to create a new column "id_product", consisting of sequential integers that start with 1. In addition, I'd like to rename all the columns name into snake_case format.

In [6]:
import pandas as pd
df = pd.read_json("../data/C&F_sale_fragrances_cleaned.json")

# create  new column as an identifier
df['id_product'] = range(1,len(df)+1)

# rename columns name
df = df.rename(columns={'Brand':'brand',
                        'Product Name':'product_name',
                        'Size (ml)':'size_ml',
                        'Type':'type',
                        'Old Price (IDR)':'old_price_idr',
                        'Special Price (IDR)':'special_price_idr',
                        'Discount Percentage':'discount_percentage'})

# reorder the columns
df = df[['id_product',
         'brand',
         'product_name',
         'size_ml',
         'type',
         'old_price_idr',
         'special_price_idr',
         'discount_percentage']]

df.head()

,id_product,brand,product_name,size_ml,type,old_price_idr,special_price_idr,discount_percentage
0,1,LOUIS VAREL,Louis Varel My Dream EDP 100 ml,100,EDP,620000,434000,30
1,2,KORLOFF,Miss Korloff EDP 88 ml,88,EDP,1690000,1521000,10
2,3,HUGO BOSS,Hugo Boss Scent Le Parfum Men 100 ml,100,EDP,1680000,1293600,23
3,4,OULLU,Oullu Umbra EDP 15 ml,15,EDP,189000,160650,15
4,5,CRISTIANO RONALDO,Cristiano Ronaldo CR7 Origins EDT 100 ml,100,EDT,1180000,826000,30


In [7]:
df.tail()

,id_product,brand,product_name,size_ml,type,old_price_idr,special_price_idr,discount_percentage
225,226,MAUBOUSSIN,Mauboussin Private Club EDP 100 ml,100,EDP,1320000,1122000,15
226,227,SALVADOR DALI,Salvador Dali Eau De Rubylips EDT 100 ml,100,EDT,1130000,904000,20
227,228,ARMAND BASI,Armand Basi Night Blue EDT 100 ml,100,EDT,1290000,1032000,20
228,229,HUGO BOSS,Hugo Boss Selection EDT 90 ml,90,EDT,1150000,885500,23
229,230,HUGO BOSS,Hugo Boss Nuit EDP 75 ml,75,EDP,1480000,1139600,23


In [8]:
df.to_json("../data/C&F_sale_fragrances_cleaned.json", orient="records", indent=4)